# Suchen und Prüfen von definierenden Relationen

### Imports der benötigten Module

In [177]:
%run Module_Polynomdivision.ipynb

importing notebook from Polynomdivision.ipynb
importing notebook from FpRechner.ipynb


In [178]:
import numpy as np
import random
from IPython.display import display, Math

In [179]:
import Polynomdivision as polydiv
import FpRechner as Fp

### Hilfsfunktionen

#### Alle Polynome

$all\_relations(p,l)$ gibt alle Relationen $\alpha^l=r_{l-1}\alpha^{l-1}+r_{l-2}\alpha^{l-2}+\cdots+r_1\alpha+r_0$ als Tupel ($r_{l-1}$,$r_{l-2}$,$\cdots$,$r_1$,$r_0$) aus, die für den Erweiterungskörper $\mathbb F_q$ mit $q=p^l$ als definierende Relation in Frage kommen.
Hierbei sind die Relationen mit $r_0=0$ bereits unberücksichtigt, da deren Polynome Nullstellen haben. 

In [232]:
def all_relations(p,l):
    menge = set()
    for i in range(pow(p,l)):
        if i%p !=0:
            result=(i%p,)
            for j in range (1,l):
                result = ((i//pow(p,j)%p),)+result
            menge.add(result)
    return menge

$nullstellenfreie(p,l)$ gibt alle Relationen der Funktion $all\_relations(p,l)$ zurück, deren Polynome keine Nullstelle haben.

In [233]:
def nullstellenfreie(p,l):
    zerozeros=[]
    allpol = all_relations(p,l)
    
    for element in allpol:
        check=True
        
        for i in range(p):
            test = pow(i,l)
        
            for j in range(l):
                test = test - element[j]*pow(i,l-j-1)
                            
            if test%p == 0:
                check = False
                break;
        
        if check == True:
            zerozeros.append(element)
    
    return zerozeros

$relation\_to\_polynom(rel,p)$ wandelt $\alpha^l=r_{l-1}\alpha^{l-1}+r_{l-2}\alpha^{l-2}+\cdots+r_1\alpha+r_0$ in $X^l-r_{l-1}X^{l-1}-r_{l-2}X^{l-2}-\cdots-r_1X-r_0$ \
Als Tupel $rel=$($r_{l-1}$,$r_{l-2}$,$\cdots$,$r_1$,$r_0$) in ($1$,$-r_{l-1}$,$-r_{l-2}$,$\cdots$,$-r_1$,$-r_0$) 

In [225]:
def relation_to_polynom(rel,p):
    result=(1,)
    for i in range(len(rel)):
        result = result+((-rel[i])%p,)
    return result

$polynom\_to\_relation(pol,p)$ wandelt $X^l-r_{l-1}X^{l-1}-r_{l-2}X^{l-2}-\cdots-r_1X-r_0$ in $\alpha^l=r_{l-1}\alpha^{l-1}+r_{l-2}\alpha^{l-2}+\cdots+r_1\alpha+r_0$ \
Als Tupel $pol=$($1$,$-r_{l-1}$,$-r_{l-2}$,$\cdots$,$-r_1$,$-r_0$) in ($r_{l-1}$,$r_{l-2}$,$\cdots$,$r_1$,$r_0$) 

In [234]:
def polynom_to_relation(pol,p):
    result = ()
    for i in range(1,len(pol)):
        result = result+(-pol[i]%p,)
    return result

$def\_rel\_find(p,l)$ gibt alle definierenden Relationen $\alpha^l=r_{l-1}\alpha^{l-1}+r_{l-2}\alpha^{l-2}+\cdots+r_1\alpha+r_0$ für einen Körper $\mathbb F_q$ mit $q=p^l$ als eine Liste von Tupeln   ($r_{l-1}$,$r_{l-2}$,$\cdots$,$r_1$,$r_0$) aus.   

In [246]:
def def_rel_find(p,l):
    nullstellenfrei=nullstellenfreie(p,l)
    if (l==2 or l==3):
        result = nullstellenfrei
    else:
        result=[]
        menge =[]
        for i in range(2,l//2+1):
            menge = menge + def_rel_find(p,i)
        for polynom in nullstellenfrei:
            check=True
            for element in menge:
                ergebnis, rest = polydiv.poldiv(relation_to_polynom(polynom,p),relation_to_polynom(element,p),p)
                if np.sum(rest)==0:
                    check=False;
                    break;
            if check==True:
                result.append(polynom)
    result.sort()
    return result

$def\_rel\_check(rel,p)$ prüft, ob eine Relation als Tupel $rel$ = ($r_{l-1}$,$r_{l-2}$,$\cdots$,$r_1$,$r_0$) eine definierende Relation für einen Erweiterungskörper von $\mathbb F_p$ ist.

In [228]:
def def_rel_check(rel,p):
    l = len(rel)
    for i in range(p):
        test = pow(i,l)
        for j in range(l):
                test = test - rel[j]*pow(i,l-j-1)
        if test%p == 0:
            return False
    if l == 2 or l == 3:
        return True
    minpols = def_rel_find(p,l//2)
    pol = relation_to_polynom(rel,p) 
    for element in minpols:
        ergebnis, rest = polydiv.poldiv(pol,relation_to_polynom(element,p),p)
        if np.sum(rest) == 0:
            return False
    return True
    
        

$print\_minpolynom(minpolynom)$ erhält ein Minimalpolynom als Tupel $minpolynom=$($1$,$-r_{l-1}$,$-r_{l-2}$,$\cdots$,$-r_1$,$-r_0$) und stellt es in der Form $X^l-r_{l-1}X^{l-1}-r_{l-2}X^{l-2}-\cdots-r_1X-r_0$ dar.  

In [267]:
def print_minpolynom(minpolynom):
    l = len(minpolynom)
    s = r'X^{'+str(l-1)+r'}+'
    for i in range(1,l-2):
        if minpolynom[i]>1:
            s = s+(r''+str(minpolynom[i])+r'X^{'+str(l-i-1)+r'}+')
        elif minpolynom[i]==1:
            s = s+(r'X^{'+str(l-i-1)+r'}+')
    if minpolynom[l-2]>1:
        s=s+(r''+str(minpolynom[l-2])+r'X+')
    elif minpolynom[l-2]==1:
        s=s+(r'X+')
    if minpolynom[l-1]>0:
        s=s+(r''+str(minpolynom[l-1])+r'')
    else:
        s=s[:-1]
    display(Math(s))

$print\_def\_rel(def\_rel)$ erhält eine definierende Relation als Tupel $def\_rel=$($r_{l-1}$,$r_{l-2}$,$\cdots$,$r_1$,$r_0$) und stellt sie in der Form $\alpha^l=r_{l-1}\alpha^{l-1}+r_{l-2}\alpha^{l-2}+\cdots+r_1\alpha+r_0$ dar.

In [266]:
def print_def_rel(def_rel):
    l = len(def_rel)
    s = r' \alpha^{'+str(l)+r'}='
    for i in range(l-2):
        if def_rel[i]>1:
            s = s+(r''+str(def_rel[i])+r' \alpha^{'+str(l-i-1)+'}+')
        elif def_rel[i]==1:
            s = s+(r' \alpha^{'+str(l-i-1)+r'}+')
    if def_rel[l-2]>1:
        s = s+(r''+str(def_rel[l-2])+r' \alpha+')
    elif def_rel[l-2]==1:
        s = s+(r' \alpha+')
    if def_rel[l-1]>0:
        s=s+(r''+str(def_rel[l-1])+r'')
    else:
        s=s[:-1]
    display(Math(s))